In [7]:
import torch
import torch.nn as nn
import torchaudio
from transformers import Wav2Vec2Model, Wav2Vec2Config
import soundfile as sf
import numpy as np

# 1. Определение модели Few-Shot генератора
class FewShotGenerator(nn.Module):
    def __init__(self):
        super().__init__()

        # Инициализация Wav2Vec2 с правильной конфигурацией
        config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base-960h")
        self.encoder = Wav2Vec2Model(config)

        # Загрузка предобученных весов
        state_dict = torch.hub.load_state_dict_from_url(
            "https://huggingface.co/facebook/wav2vec2-base-960h/resolve/main/pytorch_model.bin"
        )
       # self.encoder.load_state_dict(state_dict)

        # Адаптер
        self.adapter = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

        # Простой генератор (в реальности используется диффузионная модель)
        self.generator = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 16000)
        )

    def forward(self, x):
        # Нормализация входа
        x = (x - x.mean()) / (x.std() + 1e-8)

        # Извлечение признаков
        outputs = self.encoder(x.unsqueeze(0))
        features = outputs.last_hidden_state.mean(dim=1)

        # Адаптация
        sound_profile = self.adapter(features)

        # Генерация
        return self.generator(sound_profile)

# 2. Инициализация модели
model = FewShotGenerator()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 3. Подготовка данных
def load_audio_samples(num_samples=3, duration=1.0, sr=16000):
    """Генерация примеров звуков (в реальности загружаются из файлов)"""
    samples = []
    for i in range(num_samples):
        # Генерация тона с разной частотой
        freq = 220 + i * 100  # 220 Гц, 320 Гц, 420 Гц
        t = torch.linspace(0, duration, int(sr * duration))
        sample = 0.5 * torch.sin(2 * np.pi * freq * t)
        samples.append(sample)
    return samples

samples = load_audio_samples()

# 4. Обучение (быстрая адаптация)
for epoch in range(10):
    epoch_loss = 0
    for sample in samples:
        optimizer.zero_grad()

        # Добавляем немного шума для разнообразия
        noisy_sample = sample + 0.01 * torch.randn_like(sample)

        output = model(noisy_sample)
        loss = nn.MSELoss()(output, sample)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(samples):.4f}")

# 5. Генерация нового звука
with torch.no_grad():
    # Используем один из примеров для контекста
    input_audio = samples[0]
    generated = model(input_audio)

    # Сохранение результата
    sf.write("generated.wav", generated.numpy(), 16000)
    print("Аудио сохранено как generated.wav")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16000])) that is different to the input size (torch.Size([1, 16000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 0.1286
Epoch 2, Loss: 0.1369
Epoch 3, Loss: 0.1156
Epoch 4, Loss: 0.1183
Epoch 5, Loss: 0.1048
Epoch 6, Loss: 0.1031
Epoch 7, Loss: 0.1295
Epoch 8, Loss: 0.0900
Epoch 9, Loss: 0.1067
Epoch 10, Loss: 0.0949
Epoch 11, Loss: 0.0909
Epoch 12, Loss: 0.0882
Epoch 13, Loss: 0.0915
Epoch 14, Loss: 0.1180
Epoch 15, Loss: 0.0900
Epoch 16, Loss: 0.0909
Epoch 17, Loss: 0.0881
Epoch 18, Loss: 0.0867
Epoch 19, Loss: 0.0873
Epoch 20, Loss: 0.0873
Epoch 21, Loss: 0.0836
Epoch 22, Loss: 0.0860
Epoch 23, Loss: 0.0897
Epoch 24, Loss: 0.0932
Epoch 25, Loss: 0.0862
Epoch 26, Loss: 0.0911
Epoch 27, Loss: 0.0843
Epoch 28, Loss: 0.0846
Epoch 29, Loss: 0.0840
Epoch 30, Loss: 0.0867
Epoch 31, Loss: 0.0883
Epoch 32, Loss: 0.0855
Epoch 33, Loss: 0.0848
Epoch 34, Loss: 0.0845
Epoch 35, Loss: 0.0861
Epoch 36, Loss: 0.0844
Epoch 37, Loss: 0.0843
Epoch 38, Loss: 0.0842
Epoch 39, Loss: 0.0841
Epoch 40, Loss: 0.0852
Epoch 41, Loss: 0.0847
Epoch 42, Loss: 0.0846
Epoch 43, Loss: 0.0842
Epoch 44, Loss: 0.09

LibsndfileError: Error opening 'generated.wav': Format not recognised.

In [10]:
with torch.no_grad():
    # Используем один из примеров для контекста
    input_audio = samples[0]
    generated = model(input_audio)
    print(generated.shape)
    torchaudio.save(
    "generated.wav",
    generated,
    sample_rate=16000,  # Частота дискретизации
    bits_per_sample=16  # 16 бит (стандарт для WAV)
)
"""    # Сохранение результата
    sf.write("generated.wav", generated.numpy(), 16000)
    print("Аудио сохранено как generated.wav")"""

torch.Size([1, 16000])


'    # Сохранение результата\n    sf.write("generated.wav", generated.numpy(), 16000)\n    print("Аудио сохранено как generated.wav")'